In [1]:
from aurelio.plotter import plot, plot_single_fold
from plotly.offline import init_notebook_mode
import os

In [2]:
init_notebook_mode(connected=True)

### Curvas de Aprendizado utilizando K-fold com ELMo, Word Embeddings de 600 dimensões e respostas adicionais para treino e dev.

In [3]:
metrics_elmo_path = "{}/metrics/elmo".format(os.getcwd())
plot(metrics_elmo_path)
# metrics_elmo_path

### Curvas de Aprendizado utilizando K-fold sem ELMo, Word Embeddings de 600 dimensões e respostas adicionais para treino e dev.

In [4]:
metrics_no_elmo_path = "{}/metrics/no_elmo".format(os.getcwd())
plot(metrics_no_elmo_path)

### Curvas de Aprendizado sem K-Fold e com ELMo, com Word Embeddings de 600 dimensões e respostas adicionais para treino e dev.

In [5]:
metrics_single_fold_elmo_path = "{}/metrics/single_fold_elmo".format(os.getcwd())
plot_single_fold(metrics_single_fold_elmo_path)

### Curvas de Aprendizado sem K-Fold e sem ELMo, com Word Embeddings de 600 dimensões e respostas adicionais para treino e dev.

In [6]:
metrics_single_fold_no_elmo_path = "{}/metrics/single_fold_no_elmo".format(os.getcwd())
plot_single_fold(metrics_single_fold_no_elmo_path)

### Curvas de Aprendizado sem K-Fold e com ELMo, com Word Embeddings de 600 dimensões e respostas adicionais apenas para dev.

In [7]:
metrics_single_fold_elmo_reduced_path = "{}/metrics/single_fold_elmo_reduced".format(os.getcwd())
plot_single_fold(metrics_single_fold_elmo_reduced_path)

### Curvas de Aprendizado sem K-Fold e sem ELMo, com Word Embeddings de 600 dimensões e respostas adicionais apenas para dev.

In [8]:
metrics_single_fold_no_elmo_reduced_path = "{}/metrics/single_fold_no_elmo_reduced".format(os.getcwd())
plot_single_fold(metrics_single_fold_no_elmo_reduced_path)